In [1]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 22
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/22
## --- Day 22: Sand Slabs ---


Enough sand has fallen; it can finally filter water for Snow Island.


Well, **almost** .


The sand has been falling as large compacted **bricks** of sand, piling up to form an impressive stack here near the edge of Island Island. In order to make use of the sand to filter water, some of the bricks will need to be broken apart - nay, **disintegrated** - back into freely flowing sand.


The stack is tall enough that you'll have to be careful about choosing which bricks to disintegrate; if you disintegrate the wrong brick, large portions of the stack could topple, which sounds pretty dangerous.


The Elves responsible for water filtering operations took a **snapshot of the bricks while they were still falling** (your puzzle input) which should let you work out which bricks are safe to disintegrate. For example:


```
 1,0,1~1,2,1
 0,0,2~2,0,2
 0,2,3~2,2,3
 0,0,4~0,2,4
 2,0,5~2,2,5
 0,1,6~2,1,6
 1,1,8~1,1,9

```


Each line of text in the snapshot represents the position of a single brick at the time the snapshot was taken. The position is given as twox,y,z``coordinates - one for each end of the brick - separated by a tilde (~``). Each brick is made up of a single straight line of cubes, and the Elves were even careful to choose a time for the snapshot that had all of the free-falling bricks at **integer positions above the ground** , so the whole snapshot is aligned to a three-dimensional cube grid.


A line like2,2,2~2,2,2``means that both ends of the brick are at the same coordinate - in other words, that the brick is a single cube.


Lines like0,0,10~1,0,10``or0,0,10~0,1,10``both represent bricks that are **two cubes** in volume, both oriented horizontally. The first brick extends in thex``direction, while the second brick extends in they``direction.


A line like0,0,1~0,0,10``represents a **ten-cube brick** which is oriented **vertically** . One end of the brick is the cube located at0,0,1``, while the other end of the brick is located directly above it at0,0,10``.


The ground is atz=0``and is perfectly flat; the lowestz``value a brick can have is therefore1``. So,5,5,1~5,6,1``and0,2,1~0,2,5``are both resting on the ground, but3,3,2~3,3,3``was above the ground at the time of the snapshot.


Because the snapshot was taken while the bricks were still falling, some bricks will **still be in the air** ; you'll need to start by figuring out where they will end up. Bricks are magically stabilized, so they **never rotate** , even in weird situations like where a long horizontal brick is only supported on one end. Two bricks cannot occupy the same position, so a falling brick will come to rest upon the first other brick it encounters.


Here is the same example again, this time with each brick given a letter so it can be marked in diagrams:


```
 1,0,1~1,2,1   <- A
 0,0,2~2,0,2   <- B
 0,2,3~2,2,3   <- C
 0,0,4~0,2,4   <- D
 2,0,5~2,2,5   <- E
 0,1,6~2,1,6   <- F
 1,1,8~1,1,9   <- G

```


At the time of the snapshot, from the side so thex``axis goes left to right, these bricks are arranged like this:


```
  x
 012
 .G. 9
 .G. 8
 ... 7
 FFF 6
 ..E 5 z
 D.. 4
 CCC 3
 BBB 2
 .A. 1
 --- 0

```


Rotating the perspective 90 degrees so they``axis now goes left to right, the same bricks are arranged like this:


```
  y
 012
 .G. 9
 .G. 8
 ... 7
 .F. 6
 EEE 5 z
 DDD 4
 ..C 3
 B.. 2
 AAA 1
 --- 0

```


Once all of the bricks fall downward as far as they can go, the stack looks like this, where?``means bricks are hidden behind other bricks at that location:


```
  x
 012
 .G. 6
 .G. 5
 FFF 4
 D.E 3 z
 ??? 2
 .A. 1
 --- 0

```


Again from the side:


```
  y
 012
 .G. 6
 .G. 5
 .F. 4
 ??? 3 z
 B.C 2
 AAA 1
 --- 0

```


Now that all of the bricks have settled, it becomes easier to tell which bricks are supporting which other bricks:


+ BrickA``is the only brick supporting bricksB``andC``.
+ BrickB``is one of two bricks supporting brickD``and brickE``.
+ BrickC``is the other brick supporting brickD``and brickE``.
+ BrickD``supports brickF``.
+ BrickE``also supports brickF``.
+ BrickF``supports brickG``.
+ BrickG``isn't supporting any bricks.


Your first task is to figure out **which bricks are safe to disintegrate** . A brick can be safely disintegrated if, after removing it, **no other bricks** would fall further directly downward. Don't actually disintegrate any bricks - just determine what would happen if, for each brick, only that brick were disintegrated. Bricks can be disintegrated even if they're completely surrounded by other bricks; you can squeeze between bricks if you need to.


In this example, the bricks can be disintegrated as follows:


+ BrickA``cannot be disintegrated safely; if it were disintegrated, bricksB``andC``would both fall.
+ BrickB`` **can** be disintegrated; the bricks above it (D``andE``) would still be supported by brickC``.
+ BrickC`` **can** be disintegrated; the bricks above it (D``andE``) would still be supported by brickB``.
+ BrickD`` **can** be disintegrated; the brick above it (F``) would still be supported by brickE``.
+ BrickE`` **can** be disintegrated; the brick above it (F``) would still be supported by brickD``.
+ BrickF``cannot be disintegrated; the brick above it (G``) would fall.
+ BrickG`` **can** be disintegrated; it does not support any other bricks.


So, in this example,5``bricks can be safely disintegrated.


Figure how the blocks will settle based on the snapshot. Once they've settled, consider disintegrating a single brick; **how many bricks could be safely chosen as the one to get disintegrated?** 




In [2]:
from itertools import cycle
import copy

ON = HIGH = 1
OFF = LOW = 0
DEBUG = False



class Brick(object):
    def __init__(self, data):
        self.s = data[0]  # start
        self.e = data[1]  # end
        self.squares = list(self._get_squares())
        self.lowest = list(self._get_lowest_squares())
        self.topest = list(self._get_topest_squares())
        self.supported = set()
        self.supports = set()
        self.name = ""  # We give bricks a one char name, to more easily debug the test scenario
    
    def count_supports(self, counted, isFirst):
        if DEBUG: print(f"checking {self.name}")
        counted.add(self)
        if len(self.supported) > 1 and not isFirst:
            return 0, counted

        if not self.supports:
            return 1, counted
        else:
            total = 1
               
            for sup in self.supports:
                if DEBUG: print(f"seeing if we can add {sup.name}")
                if sup not in counted:                    
                    
                    sub_total, counted = sup.count_supports(counted, False) 
                    if DEBUG: print(f"and adds {sub_total}")
                    
                    total += sub_total
            return total, counted
    
    def _get_topest_squares (self):
        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        elif self.s[2] > self.e[2]:
            yield (self.e[0],self.e[1], self.s[2])
        else:
            yield (self.s[0],self.s[1], self.e[2])

    def _get_lowest_squares (self):

        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        elif self.s[2] > self.e[2]:
            yield (self.e[0],self.e[1], self.e[2])
        else:
            yield (self.s[0],self.s[1], self.s[2])

    def _get_squares (self):
        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        else:
            yield from [ (self.s[0],self.s[1], it) for it in range( sorted( [self.s[2],self.e[2]] )[0], sorted( [self.s[2],self.e[2]] )[1] + 1) ]
    
    def move_down (self):
        self.s = (self.s[0], self.s[1], self.s[2]-1)
        self.e = (self.e[0], self.e[1], self.e[2]-1)
        self.squares = list(self._get_squares())
        self.topest = list(self._get_topest_squares())
        self.lowest = list(self._get_lowest_squares())

    def move_up (self):
        self.s = (self.s[0], self.s[1], self.s[2]+1)
        self.e = (self.e[0], self.e[1], self.e[2]+1)
        self.squares = list(self._get_squares())
        self.topest = list(self._get_topest_squares())
        self.lowest = list(self._get_lowest_squares())
    
    def isgrounded(self):
        return self.s[2] == 1 or self.e[2] == 1  

    def __repr__(self):
        return self.name + " " + str(self.s) + "-" + str(self.e) 


def build_state(data):
    # We give bricks a one char name, to more easily debug the test scenario
    state = []
    char = 65
    for line in data:
        b = Brick( [list(map(int, x.split(","))) for x in line.split("~")]) 
        b.name = chr(char)
        state.append(b)
        char += 1
    return state


def others_in(state, current):
    length = len(state)
    yield from list(map(lambda x: x%length,range(current+1,current+length)))

def move_if_possible(state, squares, counter):
    brick = state[counter]
    if DEBUG: print(f"checking {brick} ")
    if DEBUG: print(f"removing {brick.squares} ")
    for s in brick.squares:
        squares.remove(s)    
    if DEBUG: print(squares)
    done = False
    moves = 0
    while not done:
        if brick.isgrounded():
            if DEBUG: print(f"{brick} is grounded")
            done = True
            break

        brick.move_down()    
        moves += 1
        if DEBUG: print(f"checking {brick.lowest}")
        for i in brick.lowest:
            if i in squares:
                if DEBUG: print(f"{i} and it is!!")
                done = True
                brick.move_up()
                moves -= 1
                break
    for s in brick.squares:
        squares.add(s)
    return moves


def it(state , current):
    length = len(state)
    print(list(map(lambda x: x%length,range(current+1,current+length))))


def settle(state):

    squares = set()
    for b in state:
        for s in b.squares:
            squares.add(s)

    idx_cycler = cycle (range(len(state)))
    total_moves = 0
    if DEBUG: print(state)
    non_progress_counter = 0

    while non_progress_counter < len(state):
        current = next(idx_cycler)
        if DEBUG: print(f"\n\n\n\t do a pass with: {current}  ")

        moves = move_if_possible(state, squares, current)
        

        total_moves += moves
        if DEBUG: print(f"\nmade {moves} moves!")
        if DEBUG: print(f"\t ================\n")
        if not moves:
            non_progress_counter += 1
        else:
            non_progress_counter = 0
    return state, total_moves

def build_cubes_to_bricks (state):
    cubes_to_Bricks = {}
    
    for brick in state:
        for square in brick.squares:
            cubes_to_Bricks[square] = brick
    
    return cubes_to_Bricks
    

def find_supports(state, cubes_to_Bricks):
    for brick in state:
        if DEBUG: print(f"\nchecking {brick.name}")
        if DEBUG: print(f"=============")
        if DEBUG: print(f"\nTopest {brick.topest}")
        if DEBUG: print(f"\nsquares {brick.squares}")
        if DEBUG: print(f"\nlowest {brick.lowest}")

        for square in brick.topest:            
            up = (square[0], square[1], square[2]+1)
            if DEBUG: print(f"{up}")
            if up in cubes_to_Bricks:
                if DEBUG: print(f"{brick.name} supports {cubes_to_Bricks[up].name}")
                brick.supports.add(cubes_to_Bricks[up])
        
        if not brick.isgrounded():
            for square in brick.lowest:
                down = (square[0], square[1], square[2]-1)
                if down in cubes_to_Bricks:
                    if DEBUG: print(f"{brick.name} is supported by {cubes_to_Bricks[down].name}")
                    brick.supported.add(cubes_to_Bricks[down])    

def solution_1 (data):
    state = build_state(data)
    state, discard = settle(state)
    print ("settled")
    cubes_to_Bricks = build_cubes_to_bricks(state)
    find_supports(state, cubes_to_Bricks)
    
    total = 0
    
    for brick in state:
        if DEBUG: print(f"brick {brick} supports: {len(brick.supports)} supported by: {len(brick.supported)}")
        s = set()
        a, b = brick.count_supports(s, True)

        if a==1:
            if DEBUG:print(f"brick {brick} can be desintegrated {a} {b}")
            total +=1
    print(total)
    return total

assert solution_1 (puzzle.test) == 5
print( f"Solution 1:  {solution_1 (puzzle.data)} bricks can be safely chosen to get disintegrated") # 418




settled
5
settled
418
Solution 1:  418 bricks can be safely chosen to get disintegrated


In [45]:
show_problem_2(puzzle)

## --- Part Two ---


Disintegrating bricks one at a time isn't going to be fast enough. While it might sound dangerous, what you really need is a **chain reaction** .


You'll need to figure out the best brick to disintegrate. For each brick, determine how many **other bricks would fall** if that brick were disintegrated.


Using the same example as above:


+ Disintegrating brickA``would cause all6``other bricks to fall.
+ Disintegrating brickF``would cause only1``other brick,G``, to fall.


Disintegrating any other brick would cause **no other bricks** to fall. So, in this example, the sum of **the number of other bricks that would fall** as a result of disintegrating each brick is7``.


For each brick, determine how many **other bricks** would fall if that brick were disintegrated. **What is the sum of the number of other bricks that would fall?** 




In [21]:
DEBUG = False
DEBUG2 = False

class Brick(object):
    def __init__(self, data):
        self.s = data[0]  # start
        self.e = data[1]  # end
        self.squares = list(self._get_squares())
        self.lowest = list(self._get_lowest_squares())
        self.topest = list(self._get_topest_squares())
        self.supported = set()
        self.supports = set()
        self.name = ""  # We give bricks a one char name, to more easily debug the test scenario



    def count_supports(self, counted, isFirst):
        if DEBUG2: print(f"checking {self.name}")
        counted.add(self)
        if len(self.supported) > 1 and not isFirst:
            return 0, counted

        if not self.supports:
            return 1, counted
        else:
            total = 1
               
            for sup in self.supports:
                if DEBUG2: print(f"seeing if we can add {sup.name}")
                if sup not in counted:                    
                    
                    sub_total, counted = sup.count_supports(counted, False) 
                    if DEBUG2: print(f"and adds {sub_total}")
                    
                    total += sub_total
            return total, counted
    
    def _get_topest_squares (self):
        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        elif self.s[2] > self.e[2]:
            yield (self.e[0],self.e[1], self.s[2])
        else:
            yield (self.s[0],self.s[1], self.e[2])

    def _get_lowest_squares (self):

        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        elif self.s[2] > self.e[2]:
            yield (self.e[0],self.e[1], self.e[2])
        else:
            yield (self.s[0],self.s[1], self.s[2])

    def _get_squares (self):
        if self.s[2] == self.e[2]:
            if self.s[0] == self.e[0]:  # Y situation
                yield from [ (self.s[0],it ,self.s[2]) for it in range( sorted( [self.s[1],self.e[1]] )[0], sorted( [self.s[1],self.e[1]] )[1] + 1) ]
            else:   # X situation
                yield from [ (it, self.s[1],self.s[2]) for it in range( sorted( [self.s[0],self.e[0]] )[0], sorted( [self.s[0],self.e[0]] )[1] + 1) ]
        else:
            yield from [ (self.s[0],self.s[1], it) for it in range( sorted( [self.s[2],self.e[2]] )[0], sorted( [self.s[2],self.e[2]] )[1] + 1) ]
    
    def move_down (self):
        self.s = (self.s[0], self.s[1], self.s[2]-1)
        self.e = (self.e[0], self.e[1], self.e[2]-1)
        self.squares = list(self._get_squares())
        self.topest = list(self._get_topest_squares())
        self.lowest = list(self._get_lowest_squares())

    def move_up (self):
        self.s = (self.s[0], self.s[1], self.s[2]+1)
        self.e = (self.e[0], self.e[1], self.e[2]+1)
        self.squares = list(self._get_squares())
        self.topest = list(self._get_topest_squares())
        self.lowest = list(self._get_lowest_squares())
    
    def isgrounded(self):
        return self.s[2] == 1 or self.e[2] == 1  

    def __repr__(self):
        return self.name #+ " " + str(self.s) + "-" + str(self.e) 


def solution_2 (data):
    
    state = build_state(data)
    state, discard = settle(state)
    print ("settled")
    cubes_to_Bricks = build_cubes_to_bricks(state)
    find_supports(state, cubes_to_Bricks)
    
    total = 0
    print("state")
    for brick in state:
        if DEBUG2: print(f"\n\nWILL FALL IF {brick.name} were dissintegrated")
        if DEBUG2: print(f"======================================")   
        total += howMany_will_fall_if_removed(brick, state)
    print(total)
    return total


def howMany_will_fall_if_removed(brick, state):

    total = 0
    keepchecking = True
    counter = 0
    removed = set()
    removed.add(brick)
    while keepchecking:

        fallen_this_round = 0
        for br in state:
            if can_brick_fall(br, removed):
                if DEBUG2:print(f"{br.name} can fall")
                fallen_this_round +=1
                total += 1
                keepchecking = True
                removed.add(br)
        
        if fallen_this_round == 0:
            keepchecking = False
        if DEBUG2:print (f"fallen_this_round {fallen_this_round}\n")
        counter += 1

    if DEBUG2: print (f"{total} would fall if {brick.name} were dissintegrated")
    return total

def can_brick_fall(brick, removed):
    if DEBUG2: print (f"{brick.name}-> {brick.supported} {removed}")
    return not brick.isgrounded() and brick not in removed and all( [s in removed for s in brick.supported])

assert solution_2 (puzzle.test) == 7
print( f"Solution 2:  {solution_2 (puzzle.data)} is the sum of the number of other bricks that would fall") # 70702 in 8.2s


settled
state
7
settled
state
70702
Solution 2:  70702 is the sum of the number of other bricks that would fall
